bert 환경에서 실행

In [27]:
from datasets import *
from transformers import *
from tokenizers import *
import re
import string
import numpy as np
import pandas as pd
import os
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [28]:
train_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/train.csv')
test_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/test.csv')

train_df.head(10)

TEXT_LABELS = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}

def combine_title_and_description(df):
    # Returns a dataset with the title and description fields combined
    df['text'] = df[['Title', 'Description']].agg('. '.join, axis=1)
    df = df.drop(['Title', 'Description'], axis=1)
    return df

train_df = combine_title_and_description(train_df)
test_df = combine_title_and_description(test_df)

#각 클래스별로 5000개씩 총 2만개의 데이터를 샘플랭(너무 크면 TextCuboid의 용량이 너무 커진다)
sampled_df = train_df.groupby("Class Index").apply(lambda x: x.sample(5000, random_state=10))

def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    #text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

#전처리 특수기호 없애기
sampled_df['text']=sampled_df['text'].apply(clean_text)

sampled_df = sampled_df.reset_index(drop=True)

train_df = sampled_df.groupby("Class Index").apply(lambda x: x.sample(4000, random_state=10))
train_idx = [x[1] for x in train_df.index]
test_df = sampled_df.drop(train_idx)

x_train=list(train_df['text'])
y_train=list(train_df['Class Index'])
x_test=list(test_df['text'])
y_test=list(test_df['Class Index'])

to_txt=x_train+x_test
y=list(y_train)+list(y_test)

In [29]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=label[:16000]
y_test=label[16000:]

In [30]:
to_txt

['uk s straw expects turkey s talks with eu to start in january britain expects the european union to start membership talks with turkey in january when luxembourg assumes the eu s rotating presidency foreign secretary jack straw said',
 'curfew eased in tense kathmandu kathmandu nepalese authorities have briefly lifted a curfew to let people carry out essential tasks after a quiet night following riots that left two dead',
 'dog released by calgary pound to new owners back with original family canadian press canadian press  calgary cp  zack the dog was home with his original family friday but its not a happy ending for everyone involved',
 'desert hospitality honor and the war outside the door one najaf familys shifting view of the fierce fighting that surrounds them',
 'philosopher of language dies philosopher jacques derrida founder of deconstructionism has died aged  of pancreatic cancer deconstructionism is a kind of critical thinking or mode of analysis',
 'russia school siege to

버트 모델 정의

In [31]:
#분류한 데이터셋으로 처음부터 학습한 BERT 모델 load
import torch
import numpy as np
from itertools import groupby
from operator import itemgetter


model_path = "C:/Users/user/Desktop/bert/AG news/pretrained-bert-768"

from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(os.path.join(model_path, "checkpoint-10000"), output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(model_path)

loading configuration file C:/Users/user/Desktop/bert/AG news/pretrained-bert-768\checkpoint-10000\config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file C:/Users/user/Desktop/bert/AG news/pretrained-bert-768\checkpoint-10000\pytorch_model.bin
Some weights of the model checkpoint at C:/Users/user/Desktop/bert/AG news/pretrained-bert-768\checkpoint-10000 were not used when i

In [32]:
text = to_txt[5]

marked_text = "[CLS] " + text + " [SEP]"

tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화

indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)

segments_ids = [1] * len(tokenized_text)

# Python list를 PyTorch tensor로 변환하기 
tokens_tensor = torch.tensor([indexed_text])
segments_tensors = torch.tensor([segments_ids])

with torch.no_grad():
    bert_model = model(tokens_tensor, segments_tensors )
    hidden_layers = bert_model[1][1:]     #13개의 값 중 첫번째 요소는 최초 임베딩 그러므로 뒤의 12개 은닉 상태만 얻자
    
token_embeddings = torch.stack(hidden_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])

token_vecs_sum = []

for token in token_embeddings:
    
    sum_vec = torch.sum(token[-4:], dim=0)
    
    token_vecs_sum.append(np.array(sum_vec))
    

token_vecs_sum=np.array(token_vecs_sum)

In [33]:
def split_continuous_integers(lst):                                      #연속된 정수리스트를 split 해주는 함수
    for k, g in groupby(enumerate(lst), lambda i_x:i_x[0]-i_x[1]):
        yield list(map(itemgetter(1), g))
        
def add_previous_number(lst):                                            #최초 서브토큰 인덱스 추가
    return [[sub_lst[0] - 1] + sub_lst for sub_lst in lst]

def bert_word_embedding(text):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화
    indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    # Python list를 PyTorch tensor로 변환하기 
    tokens_tensor = torch.tensor([indexed_text])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        bert_model = model(tokens_tensor, segments_tensors )
        hidden_layers = bert_model[1][1:]     #13개의 값 중 첫번째 요소는 최초 임베딩 그러므로 뒤의 12개 은닉 상태만 얻자
    token_embeddings = torch.stack(hidden_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
    token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
    token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])
    
    token_vecs_sum = []
    for token in token_embeddings:                #인코더의 마지막 4개의 은닉 상태를 합쳐 최종 벡터(4개 합친 것이 성능이 가장 좋음)
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(np.array(sum_vec))
    
    token_vecs_sum=np.array(token_vecs_sum)
    #서브토큰을 결합해 단어에 대한 임베딩 벡터를 얻자
    subword_indices = [i for i, token in enumerate(tokenized_text) if '##' in token]
    index_list = add_previous_number(list(split_continuous_integers(subword_indices))) #index_list는 서브토큰에 해당하는 인덱스를 한 리스트에 묶어줌
    
    new_token_vecs_sum = []
    last_index = 0

    for subword_inx_list in index_list:
        # 이전 인덱스와 현재 인덱스 그룹 사이의 벡터를 추가합니다.
        new_token_vecs_sum.extend(token_vecs_sum[last_index:subword_inx_list[0]])

        # 현재 인덱스 그룹에 해당하는 벡터의 평균을 계산하고 추가합니다.
        avg_vecs = np.mean(token_vecs_sum[subword_inx_list], axis=0)
        new_token_vecs_sum.append(avg_vecs)

        # 마지막 인덱스를 업데이트합니다.
        last_index = subword_inx_list[-1] + 1

    # 마지막 인덱스 이후의 벡터를 추가합니다.
    new_token_vecs_sum.extend(token_vecs_sum[last_index:])

    # 결과를 numpy array로 변환합니다.
    new_token_vecs_sum = np.array(new_token_vecs_sum[1:-1])
    
    return new_token_vecs_sum

In [34]:
token_vecs_sum=bert_word_embedding(text)
token_vecs_sum.shape

(47, 768)

임베딩

In [35]:
x_train=to_txt[:16000]
x_test=to_txt[16000:]

In [36]:
except_lst=[]
for i in range(len(x_train)):
    try:
        text = to_txt[i]
        embed_matrix=bert_word_embedding(text)       
        np.save('./bert_embedding/train(ag)/doc%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(39, 768)
(30, 768)
(37, 768)
(21, 768)
(27, 768)
(47, 768)
(27, 768)
(37, 768)
(32, 768)
(43, 768)
(21, 768)
(44, 768)
(25, 768)
(32, 768)
(32, 768)
(35, 768)
(30, 768)
(23, 768)
(29, 768)
(44, 768)
(39, 768)
(36, 768)
(34, 768)
(36, 768)
(37, 768)
(38, 768)
(44, 768)
(35, 768)
(32, 768)
(30, 768)
(40, 768)
(30, 768)
(34, 768)
(48, 768)
(35, 768)
(48, 768)
(34, 768)
(45, 768)
(41, 768)
(35, 768)
(49, 768)
(25, 768)
(46, 768)
(25, 768)
(40, 768)
(30, 768)
(55, 768)
(18, 768)
(26, 768)
(41, 768)
(44, 768)
(39, 768)
(40, 768)
(44, 768)
(43, 768)
(43, 768)
(51, 768)
(33, 768)
(31, 768)
(57, 768)
(39, 768)
(42, 768)
(49, 768)
(45, 768)
(38, 768)
(23, 768)
(39, 768)
(42, 768)
(42, 768)
(34, 768)
(30, 768)
(45, 768)
(65, 768)
(36, 768)
(48, 768)
(44, 768)
(39, 768)
(41, 768)
(18, 768)
(39, 768)
(39, 768)
(63, 768)
(37, 768)
(43, 768)
(46, 768)
(49, 768)
(38, 768)
(38, 768)
(23, 768)
(26, 768)
(43, 768)
(42, 768)
(33, 768)
(49, 768)
(33, 768)
(65, 768)
(40, 768)
(39, 768)
(36, 768)
(32, 768)


(31, 768)
(39, 768)
(43, 768)
(29, 768)
(39, 768)
(35, 768)
(28, 768)
(35, 768)
(43, 768)
(40, 768)
(36, 768)
(41, 768)
(23, 768)
(45, 768)
(46, 768)
(36, 768)
(40, 768)
(66, 768)
(42, 768)
(27, 768)
(41, 768)
(41, 768)
(38, 768)
(37, 768)
(31, 768)
(37, 768)
(41, 768)
(30, 768)
(39, 768)
(45, 768)
(33, 768)
(40, 768)
(29, 768)
(42, 768)
(34, 768)
(29, 768)
(34, 768)
(21, 768)
(33, 768)
(34, 768)
(37, 768)
(45, 768)
(42, 768)
(46, 768)
(43, 768)
(39, 768)
(40, 768)
(49, 768)
(24, 768)
(33, 768)
(39, 768)
(46, 768)
(22, 768)
(50, 768)
(27, 768)
(39, 768)
(35, 768)
(36, 768)
(41, 768)
(25, 768)
(21, 768)
(51, 768)
(31, 768)
(42, 768)
(34, 768)
(34, 768)
(46, 768)
(39, 768)
(29, 768)
(42, 768)
(65, 768)
(45, 768)
(37, 768)
(42, 768)
(39, 768)
(20, 768)
(32, 768)
(30, 768)
(42, 768)
(21, 768)
(38, 768)
(39, 768)
(33, 768)
(31, 768)
(37, 768)
(22, 768)
(24, 768)
(36, 768)
(40, 768)
(34, 768)
(42, 768)
(32, 768)
(38, 768)
(43, 768)
(25, 768)
(31, 768)
(46, 768)
(37, 768)
(36, 768)
(35, 768)


(49, 768)
(38, 768)
(41, 768)
(47, 768)
(39, 768)
(41, 768)
(30, 768)
(42, 768)
(43, 768)
(40, 768)
(46, 768)
(61, 768)
(38, 768)
(28, 768)
(36, 768)
(40, 768)
(15, 768)
(27, 768)
(27, 768)
(35, 768)
(27, 768)
(37, 768)
(32, 768)
(41, 768)
(49, 768)
(69, 768)
(35, 768)
(48, 768)
(55, 768)
(34, 768)
(41, 768)
(45, 768)
(32, 768)
(24, 768)
(39, 768)
(38, 768)
(35, 768)
(45, 768)
(50, 768)
(46, 768)
(41, 768)
(35, 768)
(37, 768)
(39, 768)
(49, 768)
(25, 768)
(27, 768)
(37, 768)
(26, 768)
(20, 768)
(29, 768)
(36, 768)
(37, 768)
(22, 768)
(41, 768)
(48, 768)
(32, 768)
(31, 768)
(38, 768)
(28, 768)
(39, 768)
(43, 768)
(25, 768)
(35, 768)
(17, 768)
(36, 768)
(46, 768)
(40, 768)
(37, 768)
(37, 768)
(48, 768)
(42, 768)
(21, 768)
(39, 768)
(35, 768)
(72, 768)
(41, 768)
(34, 768)
(22, 768)
(41, 768)
(44, 768)
(44, 768)
(25, 768)
(39, 768)
(27, 768)
(41, 768)
(39, 768)
(27, 768)
(30, 768)
(33, 768)
(44, 768)
(51, 768)
(35, 768)
(39, 768)
(26, 768)
(39, 768)
(33, 768)
(23, 768)
(35, 768)
(41, 768)


(42, 768)
(44, 768)
(34, 768)
(40, 768)
(56, 768)
(23, 768)
(43, 768)
(29, 768)
(47, 768)
(43, 768)
(17, 768)
(25, 768)
(37, 768)
(52, 768)
(39, 768)
(24, 768)
(40, 768)
(53, 768)
(49, 768)
(45, 768)
(30, 768)
(40, 768)
(34, 768)
(46, 768)
(36, 768)
(32, 768)
(37, 768)
(14, 768)
(39, 768)
(33, 768)
(26, 768)
(71, 768)
(52, 768)
(38, 768)
(34, 768)
(19, 768)
(45, 768)
(30, 768)
(40, 768)
(36, 768)
(34, 768)
(31, 768)
(31, 768)
(33, 768)
(30, 768)
(46, 768)
(38, 768)
(25, 768)
(38, 768)
(29, 768)
(39, 768)
(25, 768)
(40, 768)
(28, 768)
(44, 768)
(33, 768)
(40, 768)
(44, 768)
(26, 768)
(45, 768)
(28, 768)
(41, 768)
(30, 768)
(33, 768)
(46, 768)
(39, 768)
(33, 768)
(27, 768)
(26, 768)
(40, 768)
(41, 768)
(35, 768)
(46, 768)
(30, 768)
(33, 768)
(46, 768)
(29, 768)
(35, 768)
(40, 768)
(33, 768)
(25, 768)
(35, 768)
(25, 768)
(30, 768)
(43, 768)
(43, 768)
(36, 768)
(37, 768)
(41, 768)
(35, 768)
(39, 768)
(27, 768)
(36, 768)
(33, 768)
(35, 768)
(33, 768)
(45, 768)
(46, 768)
(38, 768)
(25, 768)


(38, 768)
(46, 768)
(29, 768)
(28, 768)
(28, 768)
(50, 768)
(37, 768)
(27, 768)
(34, 768)
(37, 768)
(37, 768)
(33, 768)
(39, 768)
(43, 768)
(24, 768)
(53, 768)
(45, 768)
(31, 768)
(45, 768)
(64, 768)
(22, 768)
(34, 768)
(29, 768)
(30, 768)
(43, 768)
(30, 768)
(37, 768)
(45, 768)
(40, 768)
(37, 768)
(49, 768)
(30, 768)
(41, 768)
(33, 768)
(39, 768)
(29, 768)
(31, 768)
(48, 768)
(30, 768)
(23, 768)
(33, 768)
(47, 768)
(39, 768)
(40, 768)
(40, 768)
(22, 768)
(48, 768)
(65, 768)
(41, 768)
(44, 768)
(66, 768)
(21, 768)
(37, 768)
(38, 768)
(47, 768)
(30, 768)
(49, 768)
(20, 768)
(42, 768)
(26, 768)
(34, 768)
(21, 768)
(40, 768)
(34, 768)
(32, 768)
(40, 768)
(53, 768)
(46, 768)
(33, 768)
(17, 768)
(44, 768)
(35, 768)
(37, 768)
(34, 768)
(25, 768)
(41, 768)
(32, 768)
(45, 768)
(34, 768)
(45, 768)
(33, 768)
(35, 768)
(68, 768)
(31, 768)
(39, 768)
(35, 768)
(32, 768)
(45, 768)
(39, 768)
(68, 768)
(41, 768)
(32, 768)
(44, 768)
(42, 768)
(43, 768)
(38, 768)
(38, 768)
(34, 768)
(29, 768)
(39, 768)


...

(51, 768)
(39, 768)
(25, 768)
(27, 768)
(46, 768)
(48, 768)
(36, 768)
(34, 768)
(35, 768)
(56, 768)
(16, 768)
(46, 768)
(58, 768)
(34, 768)
(45, 768)
(33, 768)
(40, 768)
(38, 768)
(34, 768)
(59, 768)
(29, 768)
(29, 768)
(48, 768)
(74, 768)
(74, 768)
(35, 768)
(25, 768)
(27, 768)
(63, 768)
(36, 768)
(37, 768)
(32, 768)
(21, 768)
(44, 768)
(50, 768)
(31, 768)
(32, 768)
(32, 768)
(27, 768)
(42, 768)
(22, 768)
(30, 768)
(42, 768)
(22, 768)
(32, 768)
(36, 768)
(24, 768)
(33, 768)
(71, 768)
(47, 768)
(31, 768)
(26, 768)
(55, 768)
(36, 768)
(40, 768)
(45, 768)
(33, 768)
(39, 768)
(57, 768)
(34, 768)
(17, 768)
(41, 768)
(33, 768)
(18, 768)
(38, 768)
(33, 768)
(43, 768)
(50, 768)
(37, 768)
(32, 768)
(37, 768)
(30, 768)
(40, 768)
(47, 768)
(42, 768)
(21, 768)
(21, 768)
(52, 768)
(34, 768)
(37, 768)
(36, 768)
(24, 768)
(42, 768)
(23, 768)
(29, 768)
(46, 768)
(36, 768)
(38, 768)
(38, 768)
(41, 768)
(28, 768)
(19, 768)
(36, 768)
(37, 768)
(55, 768)
(35, 768)
(34, 768)
(30, 768)
(45, 768)
(25, 768)


(16, 768)
(49, 768)
(33, 768)
(43, 768)
(32, 768)
(35, 768)
(28, 768)
(47, 768)
(40, 768)
(42, 768)
(32, 768)
(30, 768)
(57, 768)
(40, 768)
(40, 768)
(36, 768)
(33, 768)
(46, 768)
(45, 768)
(42, 768)
(35, 768)
(18, 768)
(37, 768)
(42, 768)
(48, 768)
(43, 768)
(28, 768)
(43, 768)
(40, 768)
(41, 768)
(31, 768)
(60, 768)
(31, 768)
(33, 768)
(43, 768)
(31, 768)
(42, 768)
(26, 768)
(27, 768)
(30, 768)
(26, 768)
(32, 768)
(34, 768)
(24, 768)
(16, 768)
(40, 768)
(32, 768)
(59, 768)
(41, 768)
(31, 768)
(44, 768)
(17, 768)
(43, 768)
(30, 768)
(34, 768)
(38, 768)
(41, 768)
(17, 768)
(64, 768)
(21, 768)
(38, 768)
(22, 768)
(34, 768)
(34, 768)
(23, 768)
(42, 768)
(46, 768)
(35, 768)
(54, 768)
(48, 768)
(12, 768)
(65, 768)
(24, 768)
(30, 768)
(18, 768)
(31, 768)
(25, 768)
(18, 768)
(14, 768)
(38, 768)
(57, 768)
(44, 768)
(17, 768)
(41, 768)
(28, 768)
(31, 768)
(44, 768)
(55, 768)
(37, 768)
(32, 768)
(32, 768)
(43, 768)
(35, 768)
(33, 768)
(38, 768)
(41, 768)
(35, 768)
(33, 768)
(27, 768)
(27, 768)


(39, 768)
(48, 768)
(40, 768)
(37, 768)
(27, 768)
(46, 768)
(38, 768)
(26, 768)
(30, 768)
(34, 768)
(30, 768)
(25, 768)
(93, 768)
(48, 768)
(25, 768)
(39, 768)
(31, 768)
(31, 768)
(31, 768)
(38, 768)
(40, 768)
(32, 768)
(43, 768)
(38, 768)
(39, 768)
(31, 768)
(39, 768)
(36, 768)
(37, 768)
(27, 768)
(32, 768)
(44, 768)
(25, 768)
(30, 768)
(34, 768)
(16, 768)
(33, 768)
(39, 768)
(35, 768)
(47, 768)
(37, 768)
(29, 768)
(47, 768)
(30, 768)
(20, 768)
(25, 768)
(38, 768)
(30, 768)
(38, 768)
(37, 768)
(29, 768)
(41, 768)
(26, 768)
(37, 768)
(43, 768)
(41, 768)
(46, 768)
(26, 768)
(42, 768)
(45, 768)
(38, 768)
(44, 768)
(37, 768)
(23, 768)
(44, 768)
(28, 768)
(32, 768)
(25, 768)
(37, 768)
(33, 768)
(29, 768)
(42, 768)
(36, 768)
(39, 768)
(37, 768)
(32, 768)
(31, 768)
(34, 768)
(52, 768)
(18, 768)
(41, 768)
(33, 768)
(28, 768)
(39, 768)
(42, 768)
(43, 768)
(42, 768)
(24, 768)
(35, 768)
(33, 768)
(24, 768)
(33, 768)
(46, 768)
(27, 768)
(77, 768)
(51, 768)
(34, 768)
(38, 768)
(19, 768)
(28, 768)


(38, 768)
(43, 768)
(39, 768)
(32, 768)
(41, 768)
(25, 768)
(37, 768)
(41, 768)
(46, 768)
(38, 768)
(45, 768)
(35, 768)
(49, 768)
(36, 768)
(39, 768)
(50, 768)
(27, 768)
(38, 768)
(110, 768)
(26, 768)
(43, 768)
(54, 768)
(40, 768)
(40, 768)
(34, 768)
(18, 768)
(20, 768)
(40, 768)
(18, 768)
(38, 768)
(24, 768)
(59, 768)
(43, 768)
(31, 768)
(34, 768)
(36, 768)
(43, 768)
(30, 768)
(34, 768)
(28, 768)
(42, 768)
(44, 768)
(24, 768)
(38, 768)
(31, 768)
(46, 768)
(46, 768)
(84, 768)
(43, 768)
(24, 768)
(38, 768)
(36, 768)
(40, 768)
(33, 768)
(39, 768)
(37, 768)
(39, 768)
(50, 768)
(27, 768)
(36, 768)
(30, 768)
(39, 768)
(38, 768)
(23, 768)
(30, 768)
(41, 768)
(33, 768)
(28, 768)
(32, 768)
(26, 768)
(35, 768)
(35, 768)
(47, 768)
(29, 768)
(35, 768)
(38, 768)
(31, 768)
(37, 768)
(34, 768)
(30, 768)
(26, 768)
(36, 768)
(21, 768)
(41, 768)
(36, 768)
(39, 768)
(20, 768)
(35, 768)
(42, 768)
(39, 768)
(24, 768)
(38, 768)
(21, 768)
(35, 768)
(32, 768)
(28, 768)
(20, 768)
(46, 768)
(40, 768)
(35, 768)

(84, 768)
(31, 768)
(23, 768)
(39, 768)
(42, 768)
(42, 768)
(32, 768)
(46, 768)
(41, 768)
(38, 768)
(40, 768)
(29, 768)
(47, 768)
(38, 768)
(30, 768)
(36, 768)
(38, 768)
(32, 768)
(27, 768)
(44, 768)
(28, 768)
(31, 768)
(38, 768)
(43, 768)
(23, 768)
(27, 768)
(37, 768)
(29, 768)
(40, 768)
(12, 768)
(31, 768)
(56, 768)
(35, 768)
(40, 768)
(22, 768)
(32, 768)
(62, 768)
(44, 768)
(43, 768)
(39, 768)
(27, 768)
(31, 768)
(43, 768)
(54, 768)
(28, 768)
(46, 768)
(29, 768)
(24, 768)
(26, 768)
(41, 768)
(35, 768)
(29, 768)
(31, 768)
(33, 768)
(34, 768)
(33, 768)
(45, 768)
(44, 768)
(29, 768)
(26, 768)
(42, 768)
(38, 768)
(40, 768)
(31, 768)
(44, 768)
(41, 768)
(24, 768)
(42, 768)
(36, 768)
(32, 768)
(35, 768)
(54, 768)
(29, 768)
(18, 768)
(95, 768)
(49, 768)
(34, 768)
(37, 768)
(44, 768)
(31, 768)
(35, 768)
(33, 768)
(42, 768)
(30, 768)
(49, 768)
(30, 768)
(63, 768)
(42, 768)
(32, 768)
(54, 768)
(62, 768)
(33, 768)
(31, 768)
(37, 768)
(122, 768)
(43, 768)
(47, 768)
(35, 768)
(41, 768)
(49, 768)

In [37]:
import pickle

print('제외된 문서의 수: ',len(except_lst))

with open('except_lst.pkl', 'wb') as file:
    pickle.dump(except_lst, file)

print("except_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  0
except_lst를 피클 파일로 저장했습니다.


In [38]:
except_test_lst=[]

for i in range(len(x_test)):
    try:
        text = to_txt[i+16000]
        embed_matrix=bert_word_embedding(text)       
        np.save('./bert_embedding/test(ag)/test%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_test_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(43, 768)
(41, 768)
(43, 768)
(41, 768)
(24, 768)
(34, 768)
(44, 768)
(41, 768)
(44, 768)
(45, 768)
(24, 768)
(40, 768)
(38, 768)
(30, 768)
(57, 768)
(47, 768)
(42, 768)
(29, 768)
(29, 768)
(41, 768)
(44, 768)
(44, 768)
(27, 768)
(33, 768)
(67, 768)
(30, 768)
(47, 768)
(42, 768)
(48, 768)
(31, 768)
(34, 768)
(41, 768)
(44, 768)
(28, 768)
(45, 768)
(30, 768)
(24, 768)
(42, 768)
(39, 768)
(42, 768)
(42, 768)
(30, 768)
(39, 768)
(45, 768)
(34, 768)
(45, 768)
(32, 768)
(35, 768)
(32, 768)
(29, 768)
(33, 768)
(26, 768)
(36, 768)
(40, 768)
(24, 768)
(32, 768)
(44, 768)
(49, 768)
(50, 768)
(45, 768)
(46, 768)
(39, 768)
(43, 768)
(54, 768)
(45, 768)
(31, 768)
(43, 768)
(38, 768)
(44, 768)
(36, 768)
(27, 768)
(37, 768)
(34, 768)
(56, 768)
(23, 768)
(35, 768)
(23, 768)
(37, 768)
(34, 768)
(45, 768)
(40, 768)
(35, 768)
(48, 768)
(25, 768)
(38, 768)
(36, 768)
(38, 768)
(14, 768)
(33, 768)
(51, 768)
(30, 768)
(29, 768)
(22, 768)
(39, 768)
(49, 768)
(33, 768)
(40, 768)
(35, 768)
(30, 768)
(31, 768)


(42, 768)
(48, 768)
(19, 768)
(28, 768)
(30, 768)
(33, 768)
(40, 768)
(34, 768)
(51, 768)
(39, 768)
(29, 768)
(36, 768)
(47, 768)
(47, 768)
(24, 768)
(46, 768)
(39, 768)
(42, 768)
(24, 768)
(35, 768)
(24, 768)
(47, 768)
(61, 768)
(51, 768)
(40, 768)
(50, 768)
(37, 768)
(46, 768)
(43, 768)
(39, 768)
(23, 768)
(33, 768)
(35, 768)
(34, 768)
(35, 768)
(38, 768)
(27, 768)
(38, 768)
(50, 768)
(26, 768)
(55, 768)
(51, 768)
(43, 768)
(29, 768)
(41, 768)
(33, 768)
(33, 768)
(37, 768)
(45, 768)
(37, 768)
(23, 768)
(31, 768)
(41, 768)
(29, 768)
(44, 768)
(35, 768)
(34, 768)
(54, 768)
(21, 768)
(43, 768)
(36, 768)
(35, 768)
(43, 768)
(42, 768)
(23, 768)
(45, 768)
(44, 768)
(33, 768)
(46, 768)
(39, 768)
(44, 768)
(33, 768)
(71, 768)
(28, 768)
(46, 768)
(38, 768)
(15, 768)
(50, 768)
(41, 768)
(23, 768)
(42, 768)
(36, 768)
(55, 768)
(41, 768)
(43, 768)
(41, 768)
(32, 768)
(25, 768)
(40, 768)
(20, 768)
(32, 768)
(25, 768)
(38, 768)
(38, 768)
(45, 768)
(40, 768)
(40, 768)
(31, 768)
(38, 768)
(40, 768)


(37, 768)
(26, 768)
(28, 768)
(42, 768)
(37, 768)
(33, 768)
(28, 768)
(40, 768)
(49, 768)
(39, 768)
(59, 768)
(34, 768)
(33, 768)
(29, 768)
(28, 768)
(30, 768)
(57, 768)
(35, 768)
(46, 768)
(37, 768)
(37, 768)
(36, 768)
(35, 768)
(34, 768)
(26, 768)
(31, 768)
(38, 768)
(45, 768)
(44, 768)
(40, 768)
(18, 768)
(40, 768)
(61, 768)
(38, 768)
(31, 768)
(34, 768)
(32, 768)
(32, 768)
(40, 768)
(43, 768)
(43, 768)
(35, 768)
(39, 768)
(30, 768)
(37, 768)
(31, 768)
(24, 768)
(24, 768)
(34, 768)
(32, 768)
(47, 768)
(47, 768)
(42, 768)
(32, 768)
(34, 768)
(34, 768)
(30, 768)
(37, 768)
(29, 768)
(33, 768)
(43, 768)
(45, 768)
(44, 768)
(41, 768)
(34, 768)
(39, 768)
(36, 768)
(53, 768)
(30, 768)
(42, 768)
(32, 768)
(32, 768)
(32, 768)
(39, 768)
(34, 768)
(39, 768)
(32, 768)
(29, 768)
(42, 768)
(41, 768)
(36, 768)
(43, 768)
(26, 768)
(45, 768)
(30, 768)
(23, 768)
(41, 768)
(30, 768)
(34, 768)
(43, 768)
(21, 768)
(41, 768)
(47, 768)
(44, 768)
(27, 768)
(37, 768)
(33, 768)
(21, 768)
(45, 768)
(37, 768)


(37, 768)
(28, 768)
(38, 768)
(21, 768)
(30, 768)
(30, 768)
(42, 768)
(32, 768)
(29, 768)
(39, 768)
(36, 768)
(38, 768)
(41, 768)
(38, 768)
(36, 768)
(40, 768)
(34, 768)
(32, 768)
(35, 768)
(31, 768)
(27, 768)
(36, 768)
(47, 768)
(32, 768)
(48, 768)
(42, 768)
(23, 768)
(40, 768)
(35, 768)
(37, 768)
(44, 768)
(38, 768)
(33, 768)
(29, 768)
(43, 768)
(50, 768)
(28, 768)
(31, 768)
(39, 768)
(34, 768)
(39, 768)
(35, 768)
(28, 768)
(27, 768)
(38, 768)
(31, 768)
(49, 768)
(39, 768)
(35, 768)
(32, 768)
(47, 768)
(29, 768)
(46, 768)
(78, 768)
(29, 768)
(30, 768)
(46, 768)
(29, 768)
(34, 768)
(43, 768)
(26, 768)
(34, 768)
(32, 768)
(39, 768)
(21, 768)
(36, 768)
(39, 768)
(33, 768)
(34, 768)
(43, 768)
(34, 768)
(46, 768)
(34, 768)
(34, 768)
(32, 768)
(29, 768)
(41, 768)
(39, 768)
(36, 768)
(45, 768)
(38, 768)
(31, 768)
(44, 768)
(33, 768)
(35, 768)
(36, 768)
(29, 768)
(29, 768)
(48, 768)
(36, 768)
(40, 768)
(41, 768)
(42, 768)
(35, 768)
(25, 768)
(36, 768)
(38, 768)
(35, 768)
(54, 768)
(38, 768)


(37, 768)
(24, 768)
(42, 768)
(39, 768)
(33, 768)
(41, 768)
(35, 768)
(34, 768)
(18, 768)
(20, 768)
(21, 768)
(55, 768)
(111, 768)
(42, 768)
(28, 768)
(38, 768)
(27, 768)
(27, 768)
(30, 768)
(33, 768)
(68, 768)
(31, 768)
(31, 768)
(23, 768)
(31, 768)
(38, 768)
(67, 768)
(35, 768)
(39, 768)
(34, 768)
(44, 768)
(28, 768)
(28, 768)
(38, 768)
(32, 768)
(31, 768)
(47, 768)
(40, 768)
(43, 768)
(16, 768)
(27, 768)
(32, 768)
(44, 768)
(45, 768)
(24, 768)
(35, 768)
(42, 768)
(24, 768)
(28, 768)
(29, 768)
(45, 768)
(39, 768)
(48, 768)
(27, 768)
(36, 768)
(40, 768)
(30, 768)
(43, 768)
(37, 768)
(44, 768)
(30, 768)
(35, 768)
(47, 768)
(16, 768)
(29, 768)
(13, 768)
(15, 768)
(42, 768)
(17, 768)
(49, 768)
(36, 768)
(37, 768)
(41, 768)
(42, 768)
(42, 768)
(39, 768)
(28, 768)
(23, 768)
(37, 768)
(31, 768)
(41, 768)
(47, 768)
(31, 768)
(43, 768)
(36, 768)
(43, 768)
(28, 768)
(16, 768)
(31, 768)
(30, 768)
(19, 768)
(51, 768)
(24, 768)
(43, 768)
(47, 768)
(39, 768)
(36, 768)
(39, 768)
(34, 768)
(40, 768)

In [39]:
import pickle

print('제외된 문서의 수: ',len(except_test_lst))

with open('except_test_lst.pkl', 'wb') as file:
    pickle.dump(except_test_lst, file)

print("except_test_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  0
except_test_lst를 피클 파일로 저장했습니다.
